In [14]:
import numpy as np 
import tensorflow as tf
import keras
from keras import utils,callbacks
from keras.models import Sequential,Model
from keras.applications.vgg16 import VGG16
from keras.layers.core import Dropout,Flatten,Dense
from keras.layers.convolutional import Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions
import PIL.Image

In [2]:
train_dir=r"D:\jupyternotebooks\prototype 1\asl_alphabet_train\asl_alphabet_train"
test_dir=r"D:\jupyternotebooks\prototype 1\asl_alphabet_test\asl_alphabet_test"

In [3]:
classes=29
batch=256
epochs=10
lnr=0.0001

In [4]:
img_gen=ImageDataGenerator(rescale=1./255,validation_split=0.1)
train=img_gen.flow_from_directory(train_dir,target_size=(64,64),subset="training")
eval=img_gen.flow_from_directory(train_dir,target_size=(64,64),subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [5]:
adam=Adam(learning_rate=lnr)
model=Sequential()
model.add(VGG16(weights='imagenet',include_top=False,input_shape=(64,64,3)))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(29,activation='softmax'))

model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
earlystopping=callbacks.EarlyStopping(monitor="val_loss",
                                      mode='min',
                                      patience=5,
                                      restore_best_weights=True)

history=model.fit(train,
                  validation_data=eval,
                  epochs=epochs,
                  shuffle=True,
                  verbose=1,
                  callbacks=[earlystopping])

Epoch 1/10
2447/2447 [==============================] - 1870s 753ms/step - loss: 0.5751 - accuracy: 0.8295 - val_loss: 0.1062 - val_accuracy: 0.9709
Epoch 2/10
2447/2447 [==============================] - 889s 363ms/step - loss: 0.0197 - accuracy: 0.9949 - val_loss: 0.1397 - val_accuracy: 0.9661
Epoch 3/10
2447/2447 [==============================] - 801s 327ms/step - loss: 0.0211 - accuracy: 0.9948 - val_loss: 0.4334 - val_accuracy: 0.9159
Epoch 4/10
2447/2447 [==============================] - 1043s 426ms/step - loss: 0.0199 - accuracy: 0.9949 - val_loss: 0.1071 - val_accuracy: 0.9780
Epoch 5/10
2447/2447 [==============================] - 1050s 429ms/step - loss: 0.0079 - accuracy: 0.9984 - val_loss: 0.1050 - val_accuracy: 0.9769
Epoch 6/10
2447/2447 [==============================] - 895s 366ms/step - loss: 0.0077 - accuracy: 0.9983 - val_loss: 0.0540 - val_accuracy: 0.9869
Epoch 7/10
2447/2447 [==============================] - 978s 400ms/step - loss: 0.0091 - accuracy: 0.9982 - v

In [9]:
model.save('D:\jupyternotebooks\prototype 1')

INFO:tensorflow:Assets written to: D:\jupyternotebooks\prototype 1\assets


In [8]:
import datetime
print(datetime.datetime.now().time())

19:01:15.058894


In [14]:
test=ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,target_size=(64,64),class_mode=None)
pred=model.predict(test[0])
pred = np.argmax(pred, axis=1)
pred

Found 28 images belonging to 1 classes.


array([ 3, 10,  0, 24, 15, 12, 20, 27, 21, 18, 11, 22, 28,  4,  6, 23,  9,
       19, 13,  1, 14,  8, 25,  7,  2, 16, 17,  5], dtype=int64)

In [22]:
model = keras.models.load_model('D:\jupyternotebooks\prototype 1')
dir=r"D:\jupyternotebooks\test"
test=ImageDataGenerator(rescale=1./255).flow_from_directory(dir,target_size=(64,64),class_mode=None)
pred=model.predict(test)
pred = np.argmax(pred, axis=1)
pred[0]

Found 1 images belonging to 1 classes.


4